# Чатботы 3

В этот раз мы разберем пример бота: [TedBot](https://telegram.me/tedcombot) - бот, который по ключевым словам находит видео с [Ted.com](https://www.ted.com/) и отправляет ссылку на него.

Что нужно, чтобы написать такого бота?

## Данные - видео с TED
Во-первых нужна информация о видео, доступных на Ted.com - в частности, ссылки на видео и их описание. Можно написать краулер, который обкачивает сайт и собирает всю информацию, но иногда жизнь оказывается проще.  В сети есть открытый и регулярно обновляемый [гуглдок](https://docs.google.com/spreadsheets/d/1Yv_9nDl4ocIZR0GXU3OZuBaXxER1blfwR_XHvklPpEM/edit?hl=en&hl=en&hl=en#gid=0), в котором уже записана вся необходимая информация: ссылка на видео, автор, заголовок, описание, тэги, дата выступления и т.д. 

Скачиваем гуглдок в формате tsv (tab separated values) и обрабатываем его, как нам нужно. Например, нам удобно хранить всю информацию в виде json, так что сохраним всю информацию в файл `database.json`.

In [ ]:
"""
Read datatable and create a json file.
"""
import json

with open('database.tsv', 'r', encoding='utf-8') as f:
    d = {}
    talk_id = 1
    for line in f.readlines()[1:]:
        line = line.split('\t')
        d[talk_id] = {}
        d[talk_id]['URL'] = line[1]
        d[talk_id]['speaker_name'] = line[2]
        d[talk_id]['headline'] = line[3]
        d[talk_id]['description'] = line[4]
        d[talk_id]['event'] = line[5]
        d[talk_id]['duration'] = line[6]
        d[talk_id]['primary_language'] = line[7]
        d[talk_id]['published'] = line[8]
        d[talk_id]['tags'] = line[9].rstrip().split(',')
        talk_id += 1

with open('database.json', 'w', encoding='utf-8') as f2:
    json.dump(d, f2, indent=2)


## Работа с API - извлечение ключевых слов
Во-вторых, нужно как-то находить ключевые слова. Идея состоит в том, чтобы пользователь написал что-то вроде "Hey, I'd like to watch something inspirational about language or linguistics" и ему в ответ приходит ссылка на подходящее видео. Извлечение ключевых слов в тексте - вполне лингвистическая задача, и мы можем написать код, который это делает. А можем воспользоваться уже готовыми решениями. 

Одним из таких решений являются когнитивные сервисы от Microsoft (Microsoft Cognitive Services) - набор API для работы с готовыми машиннообученными моделями Microsoft. Например, там есть API для работы с изображениями - посылаешь картинку или видео, а в ответ приходит информация о том, что там изображено, а если там изображены люди, то сколько им лет и какие у них эмоции и т.д. Ну и еще там есть лингвистические API: обработка речи, извлечение сущностей, спеллчекинг, сентимент-анализ, перевод, ну и извлечение ключевых слов.

Как работают API мы уже с вами поняли на примере ВК и Телеграма: регистрируешься и получаешь токен доступа, посылаешь запрос по специальной ссылке и получаешь ответ в каком-то заданном виде, например, json. Когнитивные сервисы работают точно так же.

Чтобы получить токен, нужно зарегистрироваться: [например, тут](https://azure.microsoft.com/en-us/try/cognitive-services/) найти Text Analytics API, нажать Create и зарегистрироваться (можно использовать свой аккаунт на GitHub, это занимает 1 минуту). После этого вы получаете ключ для бесплатной работы с Text Analytics API сроком на 90 дней. 

Когда у нас есть ключ, можно работать с API. В коде ниже первая функция `get_key_words` собирает post-запрос, вместо того чтобы включать параметры запроса в саму ссылку, мы передаем запрос в виде json. Вторая функция `process_request` собственно отправляет запрос и получает ответ: если код 200 - возвращает данные, если нет - печатает код ошибки и возвращает None.

In [ ]:
def get_key_words(text):
    """
    Get key phrases from text using Cognitive Services API.
    Args:
        text: string with a text in English
    Returns:
        a list of key phrases or None
    """
    headers = dict()
    headers['Ocp-Apim-Subscription-Key'] = config.KEYWORDS_KEY   # config.KEYWORDS_KEY stands for YOUR API KEY
    headers['Content-Type'] = 'application/json'
    params = None
    json_data = {
        "documents": [
            {
                "language": "en",
                "id": 'string',
                "text": text
            }
        ]
    }
    data = None
    time.sleep(random.choice([1, 2, 3, 4, 5, 6, 8, 10, 17, 15, 20]))
    
    # config.KEYWORDS_URL stands for 'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/keyPhrases'
    # this is the URL where we send our request
    result = process_request('post', config.KEYWORDS_URL, json_data, data, headers, params)
    if result and 'documents' in result:
        return result['documents'][0]["keyPhrases"]
    return None


def process_request(method, url, json, data, headers, params):
    """
    Helper function to process the request
    """
    result = None
    response = requests.request(method, url, json=json, data=data, headers=headers, params=params)

    if response.status_code == 200:

        if 'content-length' in response.headers and int(response.headers['content-length']) == 0:
            result = None
        elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str):
            if 'application/json' in response.headers['content-type'].lower():
                result = response.json() if response.content else None

    else:
        print("Error code: %d" % (response.status_code))
        print(response.json())

    return result

Дополнительно можно посмотреть:
* [Демо извлечения ключевых слов с API](https://azure.microsoft.com/ru-ru/services/cognitive-services/text-analytics/)
* [Quick Start](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quick-start) 
* [описание API](https://westus.dev.cognitive.microsoft.com/docs/services/TextAnalytics.V2.0/operations/56f30ceeeda5650db055a3c6)


## Вспомогательные файлы и функции
Теперь, когда мы умеем находить ключевые слова и у нас есть данные, можем писать бота. Начнем с файла `config.py`, в котором будут храниться токены:

In [ ]:
TOKEN = '==='  # токен бота от BotFather

# это пути к файлам, которые мы будем использовать
DATA = '/data/database.json'  # база видео
TAGLIST = '/data/taglist.txt'  # список тэгов к видео

KEYWORDS_URL = 'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/keyPhrases'  # ссылка, куда мы отправляем запрос
KEYWORDS_KEY = '==='  # ключ к Text Analytics API 

WEBHOOK_HOST = '==='  # адрес нашего приложения в интернете
WEBHOOK_PORT = '443'

Напишем еще две вспомогательные штуки: класс Video и функцию, которая читает нашу базу данных. 

In [ ]:
import json
import config
import os

DIR = os.path.dirname(os.path.realpath(__file__))

# Классы мы будем проходить на 3м курсе, но сейчас достаточно понять, что наш класс работает как словарь: 
# в данном случае мы там храним те же ключи и значения, что в словаре, но записывать это можем короче
class Video:
    """
    Store data for one ted talk from database.
    """
    def __init__(self, values):
        vars(self).update(values)


def read_data():
    """
    Create an array of Video instances: one instance for each ted talk.
    Returns:
        list of Video instances
    """
    with open(DIR + config.DATA, 'r', encoding='utf-8') as f:
        database = json.load(f)
    arr = []
    for i in database:
        arr.append(Video(database[i]))
    return arr

Функция `read_data()` возвращает массив из элементов Video, у каждого из этих элементов есть атрибуты url, speaker_name, headline, description, event, duration, primary_language, published, tags. Это значит, что мы можем обращаться к этим значениям через точку: 

In [ ]:
video = read_data()[0]  # первое видео из массива
print(video.headline)  # распечатает название первого видео

## Бот

Начинаем бота. Функции `read_data` и `get_key_words` мы записали в файл `utils.py`.

Бот получает сообщение и отправляет его в функцию `get_key_words`, которая возвращает массив ключевых слов, если они нашлись, или `None`. Если ключевые слова нашлись, мы отправляем их в `tag_search` и `description_search` - это две функции, которые проходят по массиву с видео и проверяют, есть ли ключевые слова среди тэгов или среди описания. Ну и наконец результат попадает в функцию `send_video`, которая собирает сообщение пользователю. 

In [ ]:
# -*- coding: utf-8 -*-
import telebot
import random

import config

from utils import read_data, get_key_words
from collections import defaultdict


DATABASE = read_data()  # actually, not a database - just a list of Video instances

bot = telebot.TeleBot(config.TOKEN, threaded=False)


@bot.message_handler(content_types=["text"])  # reacts to any text message
def get_video_by_text(message):  
    kw = get_key_words(message.text)
    to_search = message.text
    if kw:
        to_search = ','.join(kw)
    video = tag_search(to_search)
    if video is None:
        video = description_search(to_search)
    send_video(message.chat.id, video)


def send_video(chat_id, video):
    """
    Send info about ted talk and URL to user or say that nothing was found.
    Args:
        chat_id:  id of a telegram chat
        video:  Video instance
    """
    if not video:
        bot.send_message(chat_id, 'Nothing found')
    else:
        collect_message = 'This is what I found:\n\n'
        collect_message += video.headline + '\r\nBy ' + video.speaker_name + ' \r\n\r\n'
        collect_message += 'Tags in video: ' + ', '.join(video.tags) + ' \r\n\r\n'
        collect_message += 'Description: ' + video.description + '\r\n\r\n'
        bot.send_message(chat_id, collect_message + video.URL)

    
def tag_search(q):
    """
    Find a ted-talk that has query words among its tags.
    Args:
        q: string with user query
    Returns:
        Video instance or None
    """
    q = q.lower()
    q = [i.strip() for i in q.split(',')]
    d = defaultdict(int)
    for video in DATABASE:
        for n in q:
            if n in video.tags:
                d[video] += 1
    if d:
        arr = []
        bestchoice = d[sorted(d, key=d.get, reverse=True)[0]]
        if bestchoice == 0:
            return None
        for i in d:
            if d[i] == bestchoice:
                arr.append(i)
        return random.choice(arr)


def description_search(q):
    """
    Find a ted-talk that has query words among in its description.
    Args:
        q: string with user query
    Returns:
        Video instance or None
    """
    words = [i.strip().lower() for i in q.split(',')]
    arr = []
    for video in DATABASE:
        if all(i in video.description.lower() or i in video.tags for i in words):
            arr.append(video)
    if len(arr) != 0:
        return random.choice(arr)

Хорошая идея держать все сообщения, которые бот может отправлять пользователю в отдельном файле. А еще можно придумать несколько разных реплик под одну ситуацию и выдавать каждый раз случайную. Например, когда бот что-то нашел, он может сказать "Смотри, что я нашел!" или "Надеюсь, тебе понравится вот это" или "Вот твое видео". Создадим файл `phrases.py` и будем хранить там массивы с репликами на все случаи жизни:

In [ ]:
"""
List all phrases that a bot can say to user.
"""

russian = ['Ya ne ponimau po-russki. :(',
           'Sorry, no Russian.',
           'Ghbdtn! Please use English.',
           "Do you speak English? Because I do."]

topics = ['Here are some topics that might get your attention:\n\n',
           'Maybe you will be interested in these topics:\n\n',
           'Check out some of these:\n\n',
          'I have a small set of tags for you:\n\n']

no_match = ["Sorry, no matching videos. :(",
            "Ooops, I couldn't find anything. :scream:",
            'Bad luck. Nothing found. Try something else. ',
            'Seems, TED has no video on this topic.']

found_something = ["Here's what I found!\n\n",
                   "Check this out!\n\n",
                   "Wow, this one is definitely worth watching!\n\n",
                   "How do you find this one?\n\n"]

start = "Hi! I can send you an inspirational video from ted.com. Just type any topic. If you can't choose a topic, please type /random. For more instructions type /help."

help = "You can input any key words separated by comma (e.g. 'linguistics, math'), and I'll send you a matching video. You can type questions in English as you usually do, e.g. 'Please, send me a video about space and aliens'. I'll do my best to show you relevant TED-talks. \nOr you can use our advanced search, just type your command before the query.\nHere is the list of possible commands:\n/taglist         Get 20 random topics from our tag list\n/random       Get random video.\n/tags            Search video by tags.\n/description   Search video by words from description.\n/author         Search video by author. \n\nHere are some examples:\n/tags linguistics, math\n/author Elon Musk\n/description learning, experimentation, science"

А в сам код с ботом можно импортировать этот файл и использовать вместе с модулем `random`:

In [ ]:
import phrases

@bot.message_handler(commands=['start'])
def start_command(message):  # reacts to /start command
    bot.send_message(message.chat.id, phrases.start)


@bot.message_handler(commands=['help'])
def help_command(message):  # reacts to /help command
    bot.send_message(message.chat.id, phrases.help)

def send_video(chat_id, video):
    """
    Send info about ted talk and URL to user or say that nothing was found.
    Args:
        chat_id:  id of a telegram chat
        video:  Video instance
    """
    if not video:
        bot.send_message(chat_id, random.choice(phrases.no_match))
    else:
        collect_message = random.choice(phrases.found_something)
        collect_message += video.headline + '\r\nBy ' + video.speaker_name + ' \r\n\r\n'
        collect_message += 'Tags in video: ' + ', '.join(video.tags) + ' \r\n\r\n'
        collect_message += 'Description: ' + video.description + '\r\n\r\n'
        bot.send_message(chat_id, collect_message + video.URL)


Можем дописать нашего бота, чтобы он реагировал на русские сообщения, говорил, что не понимает и посылал смешные стикеры. Чтобы послать стикер, нужно знать ID этого стикера. ID можно узнать у другого бота - https://telegram.me/GetStickerIdBot.  (Посылаем боту стикер, а он в ответ шлет нам ID этого стикера, который можно использовать в функции `bot.send_sticker(chat_id, sticker_id`.)

Создадим массив со всеми ID стикеров, которые мы хотим отправлять в файле `stickers.py`:

In [ ]:
"""
Store all stickers that a bot can send to user.
"""

fish = ['BQADAgADBAADijc4AAFx0NNqDnJm4QI', 'BQADAgADBgADijc4AAH50MoMENn2lQI', 'BQADAgADCAADijc4AAGB93daGX3cWgI',
        'BQADAgADLQADijc4AAGBowxjAqAlGwI',
        'BQADAgADDgADijc4AAGOGq6J30OGfwI', 'BQADAgADEAADijc4AAESVXqKiwYE2wI', 'BQADAgADEgADijc4AAF00GirhpifXQI',
        'BQADAgADFAADijc4AAGtl5dISqHmiAI',
        'BQADAgADFgADijc4AAErJ-ihzzsO7wI', 'BQADAgADJwADijc4AAE3oUMhargOuAI', 'BQADAgADGQADijc4AAHtT7j-b6m-2QI',
        'BQADAgADGwADijc4AAEdwByBSe9kgQI',
        'BQADAgADHQADijc4AAEw0RBgpCTPAAEC', 'BQADAgADHwADijc4AAFXWsuIC4i6fAI', 'BQADAgADMwADijc4AAGU2NZK2N9ilwI']

И добавим регулярное выражение для поиска кириллических символов:

In [ ]:
import stickers
import re
regRus = re.compile('[а-яёА-ЯЁ]+')


@bot.message_handler(content_types=["text"])
def get_video_by_text(message):  # reacts to any text message
    # if the message is in Russian, the bot says it knows no Russian and sends a sticker
    if regRus.search(message.text) is not None:
        bot.send_message(message.chat.id, random.choice(phrases.russian))  # random phrase about Russian
        bot.send_sticker(message.chat.id, random.choice(stickers.fish))  # random sticker with fish
    else:
        # get key phrases from cognitive services and search database
        kw = get_key_words(message.text)
        to_search = message.text
        if kw:
            to_search = ','.join(kw)
        video = tag_search(to_search)
        if video is None:
            video = description_search(to_search)
        send_video(message.chat.id, video)

Научим бота возвращать случайное видео:

In [ ]:
@bot.message_handler(commands=['random'])
def random_video_command(message):  # reacts to /random command - sends a random ted talk
    send_video(message.chat.id, random.choice(DATABASE))

И находить видео по автору:

In [ ]:
def author_search(q):
    """
    Find a ted-talk that has query words among in the name of the speaker..
    Args:
        q: string with user query
    Returns:
        Video instance or None
    """
    arr = []
    for video in DATABASE:
        if q.lower().strip() in video.speaker_name.lower():
            arr.append(video)
    if len(arr) != 0:
        return random.choice(arr)
    
    
@bot.message_handler(commands=['author'])
def author_command(message):  # reacts to /author command - search a ted talk of a given author
    message_text = message.text.replace('/author', '').strip()
    if not message_text:
        bot.send_message(message.chat.id, 'Please, provide the author\'s name after the command /author, e.g.:\n/author Elon Musk')
        return
    video = author_search(message_text)
    send_video(message.chat.id, video)

#### Весь код находится в репозитории [tedbot](https://github.com/religofsil/tedbot). Можно посмотреть туда и позадавать вопросы.

## Take-away message
Как и всегда в программировании, когда мы пишем бота, мы делим большую задачу на много маленьких и решаем их одна за другой. Не забываем задавать себе вопросы:

1) Боту нужны какие-то данные? 
  * Находим эти данные, парсим их и приводим в нужный вид.
  * Пишем функции для работы с этими данными.
  
2) Бот общается с каким-то API?
  * Читаем документацию к API.
  * Пишем функции для отправки запросов и получения ответов.
  
3) Бот реагирует на какие-то команды?
  * Для каждой команды пишем свою функцию.
  
4) Заметили, что несколько раз копируете и вставляете в разные функции один и тот же кусочек кода?
  * Стоит выделить его в отдельную функцию.
  
Кроме того, когда вы только разрабатываете бота, удобно использовать long polling, а когда вы уже выкладываете его на сервер для всеобщего использования, нужно включать вебхуки.